In [1]:
# Imports at the top of the Notebook for ease of use
import numpy as np
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
sess = tf.InteractiveSession()

# Convolutional neural network

In this notebook, we'll beat our previous MLP network using convolutions.

__1. The dataset__

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


__2. Helper functions__

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [5]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

__3. Network__

In [6]:
W_conv1 = weight_variable(shape=[5, 5, 1, 32])
b_conv1 = bias_variable(shape=[32])

W_conv2 = weight_variable(shape=[3, 3, 32, 64])
b_conv2 = bias_variable(shape=[64])

W_fc1 = weight_variable(shape=[7 * 7 * 64, 64])
b_fc1 = bias_variable(shape=[64])

W_fc2 = weight_variable(shape=[64, 10])
b_fc2 = bias_variable(shape=[10])

In [7]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])

In [8]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
y = tf.matmul(h_fc1, W_fc2) + b_fc2

__4. Loss__

In [9]:
loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)

optimizer = tf.train.GradientDescentOptimizer(0.001)
train_step = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

__5. Train__

In [10]:
sess.run(tf.global_variables_initializer())

In [11]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.1391


In [12]:
# Beware that this can take up to 20 minutes to run
for _ in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.991
